<a href="https://colab.research.google.com/github/yukinaga/lecture_pytorch/blob/master/lecture3/autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 自動微分
自動微分により、ある値の微小変化が結果に与える影響を自動で計算することができます。   
参考: https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html


## 自動微分の開始
Tensorは、requires_grad属性をTrueに設定することで計算過程が記録されるようになります。

In [0]:
import torch

x = torch.ones(2, 3, requires_grad=True)
print(x)

## Tensorの演算と自動微分
requires_grad属性がTrueであれば、演算によりgrad_fnが記録されます。  
grad_fnは、このTensorを作った演算です。  
以下では、`x`に足し算を行って得られた`y`のgrad_fnを表示します。

In [0]:
y = x + 2
print(y)
print(y.grad_fn)

掛け算、mean関数などの演算も、grad_fnに記録されます。

In [0]:
z = y * 3
print(z)

out = z.mean()
print(out)

## 勾配の計算
backwardメソッドは、逆伝播により勾配を計算します。  
その際に、 記録されている演算と経路が使用されます。  
以下の例では、aに2をかけてbとしていますが、backwardによりaの変化に対するbの変化の割合、すなわち勾配が計算されます。  


In [0]:
a = torch.tensor([1.0], requires_grad=True)
b = a * 2  # bの変化量はaの2倍
b.backward()  # 逆伝播
print(a.grad)  # aの勾配（aの変化に対するbの変化の割合）　　

より複雑な経路を持つ演算でも、backwardにより勾配を計算することができます。

In [0]:
def calc(a):
    b = a*2 + 1
    c = b*b 
    d = c/(c + 2)
    e = d.mean()
    return e

x = [1.0, 2.0, 3.0]
x = torch.tensor(x, requires_grad=True)
y = calc(x)
y.backward()
print(x.grad.tolist())  # xの勾配（xの各値の変化に対するyの変化の割合）　　

cの各値付近における勾配が計算できました。  
勾配が正しく計算できていることを確認しましょう。  
`x`を微小変化させて、`x`の微小変化に対する`y`の微小変化の割合を求めます。


In [0]:
delta = 0.001  #xの微小変化

x = [1.0, 2.0, 3.0]
x = torch.tensor(x, requires_grad=True)
y = calc(x).item()

x_1 = [1.0+delta, 2.0, 3.0]
x_1 = torch.tensor(x_1, requires_grad=True)
y_1 = calc(x_1).item()

x_2 = [1.0, 2.0+delta, 3.0]
x_2 = torch.tensor(x_2, requires_grad=True)
y_2 = calc(x_2).item()

x_3 = [1.0, 2.0, 3.0+delta]
x_3 = torch.tensor(x_3, requires_grad=True)
y_3 = calc(x_3).item()

# 勾配の計算
grad_1 = (y_1 - y) / delta
grad_2 = (y_2 - y) / delta
grad_3 = (y_3 - y) / delta

print(grad_1, grad_2, grad_3)

`x`の微小変化を0.001という小さい値にしましたが、`y`の微小変化との割合は　backwardによる計算結果とほぼ同じになりました。  
backwardにより正しく勾配を計算できていることが確認できました。